In [1]:
import ipywidgets as widgets
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import geemap
import ee

In [2]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [3]:
Map = geemap.Map()
geometry = ee.Geometry.Point([77.42174899777628,23.205021037202645])
image = ee.ImageCollection("COPERNICUS/S2_SR") \
    .filterBounds(geometry) \
    .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .first()
start_date, end_date = "2015-01-01", "2022-12-01"

ndwi = image.normalizedDifference(['B3', 'B8'])

ndwiMasked = ndwi.updateMask(ndwi.gte(0.4))
ndwiMasked1 = ndwiMasked.toInt()

vectors = ndwiMasked1.reduceToVectors(
    scale=30.0, geometryType='polygon', eightConnected=False, maxPixels=10000000, bestEffort=True)

In [4]:
geometry = ee.Geometry.Point([77.42174899777628, 23.205021037202645])
Map.addLayer(geometry)
sentinel = ee.ImageCollection("COPERNICUS/S2_SR").filterBounds(vectors) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
    .median()

visualization = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'],
}
Map.addLayer(sentinel, visualization, 'sent2rgb')
mndwi = sentinel.normalizedDifference(['B3', 'B11']).rename('mndwi')
mndwitr = mndwi.gt(0)

dissolvedoxygen = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select(
    'B9'))).add(ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen')
dissolvedoxygen2 = ee.Image(-0.0167).multiply(sentinel.select('B8')).add(ee.Image(0.0067).multiply(sentinel.select('B9'))).add(
    ee.Image(0.0083).multiply(sentinel.select('B11'))).add(ee.Image(9.577)).rename('dissolvedoxygen2').mask(mndwitr)
Map.addLayer(dissolvedoxygen2, {'min': 6.5, 'max': 8, 'palette': [
             'red', 'green', 'blue']}, 'do')

phosphate = sentinel.expression('(B4 - B5) / (B4 + B5)', {
    'B4': sentinel.select('B4'),
    'B5': sentinel.select('B5')}).rename('phosphate')
Map.addLayer(phosphate, {'min': -1, 'max': 1,'palette': ['brown', 'white', 'green']}, 'phosphate')



In [5]:
starting = '2015-01-01'
ending = '2022-12-31'

data = ee.ImageCollection(
    'COPERNICUS/S3/OLCI').filterDate(starting, ending).filterBounds(vectors)

rgb = data.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
    .median().multiply(ee.Image([0.00876539, 0.0123538, 0.0115198])).clip(vectors)

In [6]:
latlon = ee.Image.pixelLonLat().addBands(dissolvedoxygen)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100,
    tileScale=16)

data_do = np.array((ee.Array(latlon.get("dissolvedoxygen")).getInfo()))

In [8]:
latlon = ee.Image.pixelLonLat().addBands(phosphate)

latlon = latlon.reduceRegion(
    reducer=ee.Reducer.toList(),
    geometry=vectors,
    scale=100)
data_phosphate = np.array((ee.Array(latlon.get("phosphate")).getInfo()))

EEException: User memory limit exceeded.

In [ ]:
df = pd.concat([pd.DataFrame(data_do, columns=['Dissolved Oxygen']),
                pd.DataFrame(data_phosphate, columns=['Phosphate'])], axis=1, sort=False)


In [ ]:
df

,Dissolved Oxygen,Phosphate
0,0.769450,-0.166928
1,6.565650,-0.257878
2,7.906200,-0.269373
3,7.814250,-0.060146
4,8.037150,-0.065705
...,...,...
13279,0.258408,-0.089534
13280,5.248033,-0.109141
13281,10.250499,-0.111274
13282,6.282130,-0.125063
